calculate the land cover percentage in (1) Hispaniola, (2) Haiti, and (3) the Dominican Republic

In [6]:
import time
import numpy as np
import os
from os.path import join
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
import json as js
from osgeo import gdal, gdal_array, gdalconst


def percentage_calculation(df, max_lc):
    df['TOTAL'] = df.iloc[:, 1:1 + max_lc].sum(axis=1)
    
    df['1 Developed %'] = df['1 Developed'] / df['TOTAL']
    df['2 Primary wet forest %'] = df['2 Primary wet forest'] / df['TOTAL']
    df['3 Primary dry forest %'] = df['3 Primary dry forest'] / df['TOTAL']
    df['4 Secondary forest %'] = df['4 Secondary forest'] / df['TOTAL']
    df['5 Shrub/Grass %'] = df['5 Shrub/Grass'] / df['TOTAL']
    df['6 Water %'] = df['6 Water'] / df['TOTAL']
    df['7 Wetland %'] = df['7 Wetland'] / df['TOTAL']
    df['8 Other %'] = df['8 Other'] / df['TOTAL']
    
    return df


def df_pixel_count(array_lc, list_year, i_year, df):
    year = list_year[i_year]
    df.loc[i_year, 'Year'] = year
    df.loc[i_year,'1 Developed'] = np.count_nonzero(array_lc == 1)
    df.loc[i_year,'2 Primary wet forest'] = np.count_nonzero(array_lc == 2)
    df.loc[i_year,'3 Primary dry forest'] = np.count_nonzero(array_lc == 3)
    df.loc[i_year,'4 Secondary forest'] = np.count_nonzero(array_lc == 4)
    df.loc[i_year,'5 Shrub/Grass'] = np.count_nonzero(array_lc == 5)
    df.loc[i_year,'6 Water'] = np.count_nonzero(array_lc == 6)
    df.loc[i_year,'7 Wetland'] = np.count_nonzero(array_lc == 7)
    df.loc[i_year,'8 Other'] = np.count_nonzero(array_lc == 8)
    
    return df


def statis_dataframe_generate(img_lcmap, list_year, img_countrymask):
    max_lc = np.nanmax(img_lcmap)
    
    df_all = pd.DataFrame(columns=['Year', '1 Developed', '2 Primary wet forest', '3 Primary dry forest',
                                   '4 Secondary forest', '5 Shrub/Grass', 
                                   '6 Water', '7 Wetland', '8 Other'],
                              index=np.arange(0, len(list_year)))

    df_haiti = df_all.copy()
    df_dominican = df_all.copy()

    for i_year in range(0, len(list_year)):
        
        img_lcmap_eachyear = img_lcmap[i_year]
        
        img_hispaniola = img_lcmap_eachyear[(img_countrymask == 1) | (img_countrymask == 2)]
        img_haiti = img_lcmap_eachyear[img_countrymask == 1]
        img_dominican = img_lcmap_eachyear[img_countrymask == 2]

        df_all = df_pixel_count(img_hispaniola, list_year, i_year, df_all)
        df_haiti = df_pixel_count(img_haiti, list_year, i_year, df_haiti)
        df_dominican = df_pixel_count(img_dominican, list_year, i_year, df_dominican)

    df_all = percentage_calculation(df_all, max_lc)
    df_haiti = percentage_calculation(df_haiti, max_lc)
    df_dominican = percentage_calculation(df_dominican, max_lc)

    return df_all, df_haiti, df_dominican


# def main():
if __name__ == '__main__':

    pwd = os.getcwd()
    rootpath = os.path.abspath(os.path.join(pwd, '..'))
    
    list_year = np.arange(1996, 2023)
    
    img_countrymask = gdal_array.LoadFile(join(rootpath, 'data', 'hispaniola_polygon', 'countryid_hispaniola.tif'))
    
    img_lcmap = np.zeros((len(list_year), img_countrymask.shape[0], img_countrymask.shape[1]), dtype=np.uint8)
    
    for i_year in range(0, len(list_year)):
        year = list_year[i_year]
        print(year)
    
        filename_lc = join(rootpath, 'data', 'hispaniola_lc', f'hispaniola_lc_{year}.tif')
        
        img_lcmap[i_year, :, :] = gdal_array.LoadFile(filename_lc)
    

1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


In [7]:
    df_all, df_haiti, df_dominican = statis_dataframe_generate(img_lcmap, list_year, img_countrymask)

In [8]:
    df_all

,Year,1 Developed,2 Primary wet forest,3 Primary dry forest,4 Secondary forest,5 Shrub/Grass,6 Water,7 Wetland,8 Other,TOTAL,1 Developed %,2 Primary wet forest %,3 Primary dry forest %,4 Secondary forest %,5 Shrub/Grass %,6 Water %,7 Wetland %,8 Other %
0,1996,919535,3424325,616438,29664847,1733710,643966,489943,46061939,83554703,0.011005,0.040983,0.007378,0.355035,0.020749,0.007707,0.005864,0.551279
1,1997,946116,3373607,615400,29841030,1732073,644056,488004,45914417,83554703,0.011323,0.040376,0.007365,0.357144,0.02073,0.007708,0.005841,0.549513
2,1998,979072,3319014,613896,30354489,1800017,652346,481484,45354385,83554703,0.011718,0.039723,0.007347,0.363289,0.021543,0.007807,0.005763,0.542811
3,1999,1008135,3238855,613152,30665179,1890679,658848,474258,45005597,83554703,0.012066,0.038763,0.007338,0.367007,0.022628,0.007885,0.005676,0.538636
4,2000,1043009,3208257,605115,31060345,2108172,654469,478994,44396342,83554703,0.012483,0.038397,0.007242,0.371737,0.025231,0.007833,0.005733,0.531345
5,2001,1063866,3170629,604179,31196005,2077828,652904,482439,44306853,83554703,0.012733,0.037947,0.007231,0.37336,0.024868,0.007814,0.005774,0.530274
6,2002,1082076,3154164,602853,31494347,1942555,653959,485254,44139495,83554703,0.012951,0.03775,0.007215,0.376931,0.023249,0.007827,0.005808,0.528271
7,2003,1096215,3146585,601840,31696660,1920432,664790,488580,43939601,83554703,0.01312,0.037659,0.007203,0.379352,0.022984,0.007956,0.005847,0.525878
8,2004,1118880,3135441,596450,32056169,1810001,672439,490945,43674378,83554703,0.013391,0.037526,0.007138,0.383655,0.021662,0.008048,0.005876,0.522704
9,2005,1135151,3054206,593103,32499613,1866810,693918,493982,43217920,83554703,0.013586,0.036553,0.007098,0.388962,0.022342,0.008305,0.005912,0.517241


In [9]:
    df_haiti

,Year,1 Developed,2 Primary wet forest,3 Primary dry forest,4 Secondary forest,5 Shrub/Grass,6 Water,7 Wetland,8 Other,TOTAL,1 Developed %,2 Primary wet forest %,3 Primary dry forest %,4 Secondary forest %,5 Shrub/Grass %,6 Water %,7 Wetland %,8 Other %
0,1996,189618,219052,8141,7987237,331766,242434,203906,20936957,30119111,0.006296,0.007273,0.00027,0.265188,0.011015,0.008049,0.00677,0.695139
1,1997,193860,216416,8067,8020374,325012,242409,203650,20909323,30119111,0.006436,0.007185,0.000268,0.266289,0.010791,0.008048,0.006761,0.694221
2,1998,201470,213868,7990,8124732,321327,245248,203744,20800732,30119111,0.006689,0.007101,0.000265,0.269753,0.010669,0.008143,0.006765,0.690616
3,1999,210938,207945,7886,8207711,341444,245832,202196,20695159,30119111,0.007003,0.006904,0.000262,0.272508,0.011336,0.008162,0.006713,0.687111
4,2000,220214,201595,7534,8316950,366947,242663,202802,20560406,30119111,0.007311,0.006693,0.00025,0.276135,0.012183,0.008057,0.006733,0.682637
5,2001,228382,197822,7377,8342796,364675,241545,203715,20532799,30119111,0.007583,0.006568,0.000245,0.276993,0.012108,0.00802,0.006764,0.68172
6,2002,233555,196663,7057,8376663,360827,240720,203373,20500253,30119111,0.007754,0.00653,0.000234,0.278118,0.01198,0.007992,0.006752,0.680639
7,2003,236724,196125,6860,8409725,358357,241369,203544,20466407,30119111,0.00786,0.006512,0.000228,0.279216,0.011898,0.008014,0.006758,0.679516
8,2004,245218,194753,6648,8461205,355367,242738,203292,20409890,30119111,0.008142,0.006466,0.000221,0.280925,0.011799,0.008059,0.00675,0.677639
9,2005,249596,191873,6453,8559811,364573,249702,206038,20291065,30119111,0.008287,0.00637,0.000214,0.284199,0.012104,0.00829,0.006841,0.673694


In [10]:
    df_dominican

,Year,1 Developed,2 Primary wet forest,3 Primary dry forest,4 Secondary forest,5 Shrub/Grass,6 Water,7 Wetland,8 Other,TOTAL,1 Developed %,2 Primary wet forest %,3 Primary dry forest %,4 Secondary forest %,5 Shrub/Grass %,6 Water %,7 Wetland %,8 Other %
0,1996,729917,3205273,608297,21677610,1401944,401532,286037,25124982,53435592,0.01366,0.059984,0.011384,0.405677,0.026236,0.007514,0.005353,0.470192
1,1997,752256,3157191,607333,21820656,1407061,401647,284354,25005094,53435592,0.014078,0.059084,0.011366,0.408354,0.026332,0.007516,0.005321,0.467948
2,1998,777602,3105146,605906,22229757,1478690,407098,277740,24553653,53435592,0.014552,0.05811,0.011339,0.41601,0.027672,0.007618,0.005198,0.4595
3,1999,797197,3030910,605266,22457468,1549235,413016,272062,24310438,53435592,0.014919,0.056721,0.011327,0.420272,0.028993,0.007729,0.005091,0.454948
4,2000,822795,3006662,597581,22743395,1741225,411806,276192,23835936,53435592,0.015398,0.056267,0.011183,0.425623,0.032585,0.007707,0.005169,0.446069
5,2001,835484,2972807,596802,22853209,1713153,411359,278724,23774054,53435592,0.015635,0.055633,0.011169,0.427678,0.03206,0.007698,0.005216,0.44491
6,2002,848521,2957501,595796,23117684,1581728,413239,281881,23639242,53435592,0.015879,0.055347,0.01115,0.432627,0.029601,0.007733,0.005275,0.442388
7,2003,859491,2950460,594980,23286935,1562075,423421,285036,23473194,53435592,0.016085,0.055215,0.011135,0.435794,0.029233,0.007924,0.005334,0.43928
8,2004,873662,2940688,589802,23594964,1454634,429701,287653,23264488,53435592,0.01635,0.055032,0.011038,0.441559,0.027222,0.008041,0.005383,0.435374
9,2005,885555,2862333,586650,23939802,1502237,444216,287944,22926855,53435592,0.016572,0.053566,0.010979,0.448012,0.028113,0.008313,0.005389,0.429056


In [16]:
    output_excel = pd.ExcelWriter(join(rootpath, 'results', 'land_cover_pct.xlsx'))
    # output_excel.save()
    
    with pd.ExcelWriter(output_excel) as writer:  
        
        df_all.to_excel(writer, 'Hispaniola')
        df_haiti.to_excel(writer, 'Haiti')
        df_dominican.to_excel(writer, 'Dominican')
    

C:\Users\64937\AppData\Local\Temp\ipykernel_25560\2175377310.py:6: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_all.to_excel(writer, 'Hispaniola')
C:\Users\64937\AppData\Local\Temp\ipykernel_25560\2175377310.py:7: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_haiti.to_excel(writer, 'Haiti')
C:\Users\64937\AppData\Local\Temp\ipykernel_25560\2175377310.py:8: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df_dominican.to_excel(writer, 'Dominican')
